Test model dự đoán

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
import pandas as pd
import os

# ======= 1. Khởi tạo SparkSession =======
spark = SparkSession.builder.getOrCreate()

# ======= 2. Giả lập input JSON (người dùng điền quizz) =======
input_data_list = [
    {
        "Sex": 0,
        "GeneralHealth": 3,
        "HeightInMeters": 1.6,
        "WeightInKilograms": 60.0,
        "BMI": 23.4,
        "HadHeartAttack": 0,
        "HadAngina": 0,
        "HadStroke": 0,
        "HadAsthma": 0,
        "HadSkinCancer": 0,
        "HadCOPD": 0,
        "HadDepressiveDisorder": 0,
        "HadKidneyDisease": 0,
        "HadArthritis": 1,
        "DeafOrHardOfHearing": 0,
        "BlindOrVisionDifficulty": 0,
        "DifficultyConcentrating": 0,
        "DifficultyWalking": 0,
        "DifficultyDressingBathing": 0,
        "DifficultyErrands": 0,
        "SmokerStatus": 1,
        "ECigaretteUsage": 0,
        "ChestScan": 1,
        "AlcoholDrinkers": 0,
        "HIVTesting": 0,
        "FluVaxLast12": 1,
        "PneumoVaxEver": 0,
        "HighRiskLastYear": 0,
        "CovidPos": 1,
        "AgeMin": 60,
        "AgeMax": 64,
        "TetanusLast10TdapIndex": 1.0,
        "HadDiabetesIndex": 1.0
    },
    {
        "Sex": 1,
        "GeneralHealth": 2,
        "HeightInMeters": 1.7,
        "WeightInKilograms": 80.0,
        "BMI": 27.7,
        "HadHeartAttack": 0,
        "HadAngina": 0,
        "HadStroke": 0,
        "HadAsthma": 0,
        "HadSkinCancer": 0,
        "HadCOPD": 0,
        "HadDepressiveDisorder": 1,
        "HadKidneyDisease": 0,
        "HadArthritis": 0,
        "DeafOrHardOfHearing": 0,
        "BlindOrVisionDifficulty": 0,
        "DifficultyConcentrating": 1,
        "DifficultyWalking": 0,
        "DifficultyDressingBathing": 0,
        "DifficultyErrands": 0,
        "SmokerStatus": 0,
        "ECigaretteUsage": 1,
        "ChestScan": 0,
        "AlcoholDrinkers": 1,
        "HIVTesting": 1,
        "FluVaxLast12": 0,
        "PneumoVaxEver": 0,
        "HighRiskLastYear": 1,
        "CovidPos": 0,
        "AgeMin": 50,
        "AgeMax": 54,
        "TetanusLast10TdapIndex": 0.0,
        "HadDiabetesIndex": 0.0
    }
    # Thêm dòng dữ liệu khác nếu cần
]

# ======= 3. Chuyển list dict thành Pandas rồi Spark DataFrame =======
input_df = pd.DataFrame(input_data_list)
spark_df = spark.createDataFrame(input_df)

In [ ]:
# ======= 4. Load mô hình từ đường dẫn tương đối =======
model_path = "../../backend/models/Model_LogisticRegression"
model = PipelineModel.load(model_path)

In [3]:
# ======= 5. Dự đoán =======
predictions = model.transform(spark_df)

# ======= 6. Lấy kết quả dự đoán =======
results = predictions.select("probability", "prediction").collect()

print("==== KẾT QUẢ DỰ ĐOÁN NHIỀU NGƯỜI DÙNG ====\n")
for idx, row in enumerate(results):
    prob = row["probability"][1]  # Xác suất bị đột quỵ
    pred = row["prediction"]
    percent = round(prob * 100, 2)
    conclusion = "Nguy cơ CAO bị đột quỵ" if prob > 0.5 else "Nguy cơ THẤP bị đột quỵ"
    print(f"Người dùng {idx + 1}:")
    print(f"   Xác suất bị đột quỵ: {percent}%")
    print(f"   Kết luận: {conclusion}\n")


==== KẾT QUẢ DỰ ĐOÁN NHIỀU NGƯỜI DÙNG ====

Người dùng 1:
   Xác suất bị đột quỵ: 97.25%
   Kết luận: Nguy cơ CAO bị đột quỵ

Người dùng 2:
   Xác suất bị đột quỵ: 99.1%
   Kết luận: Nguy cơ CAO bị đột quỵ

